<a href="https://colab.research.google.com/github/zhangcun-yan/Traffic-safty-analyses-modeling/blob/master/Mutl_denoise_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 首先导入相关的库

In [ ]:
import scipy.io as scio
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
# 数据分析相关包
import seaborn as sns
from numpy.random import randn
import matplotlib as mpl
from scipy import stats
from matplotlib.patches import Polygon
import math
from scipy import stats

In [ ]:
# 运行后出现URL，点击获得你的认证码
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 列出你云盘里有啥
!ls "/content/drive//My Drive/data_set"

In [ ]:
mask_rcnn_orginal_total = r'/content/drive/My Drive/data_set/Mask_RCNN/wavelet_filter_Mask.csv'
mask_rcnn_orginal_remove = r'/content/drive/My Drive/data_set/Mask_RCNN/Mask_remove_wavel0926.csv'
mask_rcnn_wavelet = r'/content/drive/My Drive/data_set/Mask_RCNN/Mask_wavel0926.csv'

mask_rcnn_orginal_total = pd.read_csv(mask_rcnn_orginal_total)
mask_rcnn_orginal_total = pd.DataFrame(mask_rcnn_orginal_total)
mask_rcnn_orginal_total = mask_rcnn_orginal_total[mask_rcnn_orginal_total['Vehicle_type']==22]

mask_rcnn_orginal_remove = pd.read_csv(mask_rcnn_orginal_remove)
mask_rcnn_orginal_remove = pd.DataFrame(mask_rcnn_orginal_remove)

mask_rcnn_wavelet = pd.read_csv(mask_rcnn_wavelet)
mask_rcnn_wavelet = pd.DataFrame(mask_rcnn_wavelet)

plt.scatter(mask_rcnn_orginal_total['World_x'],mask_rcnn_orginal_total['World_y'],color='k',s=2)
plt.scatter(mask_rcnn_orginal_remove['World_x'],mask_rcnn_orginal_remove['World_y'],color='r',s=2)
# plt.scatter(mask_rcnn_remove['World_x'],mask_rcnn_remove['World_y'],color='r',s=2)
plt.show()

In [ ]:
Yolov7_orginal0 = r"/content/drive/My Drive/data_set/Yolodata/wavelet_filter_yolov7.csv"
Yolov7_orginal = r"/content/drive/My Drive/data_set/Yolodata/Yolov7_wavel0926.csv"
Yolov7_denoise = r"/content/drive/My Drive/data_set/Yolodata/Yolo_remove_wavel0926.csv"
Yolov7_orginal0 = pd.read_csv(Yolov7_orginal0)
Yolov7_orginal0 = pd.DataFrame(Yolov7_orginal0)
Yolov7_orginal0 = Yolov7_orginal0[Yolov7_orginal0['Vehicle_type']==22]

Yolov7_orginal = pd.read_csv(Yolov7_orginal)
Yolov7_orginal = pd.DataFrame(Yolov7_orginal)

Yolov7_denoise = pd.read_csv(Yolov7_denoise)
Yolov7_denoise = pd.DataFrame(Yolov7_denoise)

plt.scatter(Yolov7_orginal0['World_x'],Yolov7_orginal0['World_y'],color='r',s=2)
# plt.scatter(Yolov7_orginal['World_x'],Yolov7_orginal['World_y'],color='r',s=2)
plt.scatter(Yolov7_denoise['World_x'],Yolov7_denoise['World_y'],color='g',s=2)
plt.show()

## 导入初始数据

In [ ]:
YOLOV7_ogrinal_only_T = r"/content/drive/My Drive/data_set/Yolodata/RAW_S_T_TRAJ.xlsx"
YOLOV7_ogrinal_only_T = pd.read_excel(YOLOV7_ogrinal_only_T)
YOLOV7_ogrinal_only_T = pd.DataFrame(YOLOV7_ogrinal_only_T)
YOLOV7_ogrinal_only_T = YOLOV7_ogrinal_only_T[YOLOV7_ogrinal_only_T['Vehicle_type']==22]
plt.scatter(YOLOV7_ogrinal_only_T['World_x'],YOLOV7_ogrinal_only_T['World_y'],color='g',s=2)
plt.show()

## 导入kalman滤波后的数据

In [ ]:
YOLOV7_ogrinal = r"/content/drive/My Drive/data_set/Yolodata/S_raw.csv"
YOLOV7_ogrinal = pd.read_csv(YOLOV7_ogrinal)
YOLOV7_ogrinal = pd.DataFrame(YOLOV7_ogrinal)
YOLOV7_ogrinal = YOLOV7_ogrinal[YOLOV7_ogrinal['Vehicle_type']==22]
plt.scatter(YOLOV7_ogrinal['World_x'],YOLOV7_ogrinal['World_y'],color='g',s=2)
plt.show()

In [ ]:
YOLOV7_kalman = r"/content/drive/My Drive/data_set/Yolodata/S_Kalman.csv"
YOLOV7_kalman = pd.read_csv(YOLOV7_kalman)
YOLOV7_kalman = pd.DataFrame(YOLOV7_kalman)
YOLOV7_kalman = YOLOV7_kalman[YOLOV7_kalman['Vehicle_type']==22]
plt.scatter(YOLOV7_kalman['World_x'],YOLOV7_kalman['World_y'],color='g',s=2)
plt.show()

## 计算轨迹的相关参数

In [ ]:
# 计算轨迹中的速度/加速度/加加速度
def XY(groundtraj,caompartraj):
    g_World_x = np.array(groundtraj['World_x'].astype(float))
    g_World_y = np.array(groundtraj['World_y'].astype(float))
    com_World_x = np.array(caompartraj['World_x'].astype(float))
    com_World_y = np.array(caompartraj['World_y'].astype(float))    
    return g_World_x,g_World_y,com_World_x,com_World_y

def Velocity(trajdata):
    """recalculate the velocity, the velocity contain the X_velocity and Y-velocity and the speed"""
    """定义初速度为0"""
    len_x = trajdata.shape[0]
    Wordld_x = np.array(trajdata.World_x)
    Wordld_y = np.array(trajdata.World_y)
    velocity_x = (Wordld_x[1:len_x]-Wordld_x[0:len_x-1])/0.04
    velocity_y = (Wordld_y[1:len_x]-Wordld_y[0:len_x-1])/0.04
    velocity_x = np.insert(velocity_x,0,0)
    velocity_y = np.insert(velocity_y,0,0)
    return velocity_x,velocity_y

def Accelection(trajdata):
    """定结束时刻的加速度为0"""
    # print(trajdata)
    len_x = trajdata.shape[0]
    velocity_x = np.array(trajdata.speed_x)
    velocity_y = np.array(trajdata.speed_y)
    accelection_x = (velocity_x[1:len_x]-velocity_x[0:len_x-1])/0.04
    accelection_y = (velocity_y[1:len_x]-velocity_y[0:len_x-1])/0.04
    accelection_x = np.insert(accelection_x,0,0)
    accelection_y = np.insert(accelection_y,0,0)
    accelection_x[1] = 0
    accelection_y[1] = 0
    return accelection_x,accelection_y

def Aclculate_Jerk(trajdata):
    """计算急动度""" 
    len_x = trajdata.acc_x.shape[0]
    acc_x = np.array(trajdata.acc_x)
    acc_y = np.array(trajdata.acc_y)
    JJerk_x = (acc_x[1:len_x] - acc_x[0:(len_x-1)])/0.04
    JJerk_y = (acc_y[1:len_x] - acc_y[0:(len_x-1)])/0.04
    JJerk_x = np.insert(JJerk_x,0,0)
    JJerk_y = np.insert(JJerk_y,0,0)
    JJerk_x[2] = 0
    JJerk_y[2] = 0
    return JJerk_x,JJerk_y

## 绘制轨迹图像

In [ ]:
def Trajmap(Data,Orginal,mothod1):
    # 设置图框大小
    plt.Figure([2000,2000],dpi=20)
    fig = plt.figure(figsize=(10,5))   
    sns.despine(left=True)    
    #绘制轨迹
    plt.scatter(Data['World_x'],Data['World_y'],c='red',label='Ground trajectory')
    plt.scatter(Orginal['World_x'],Orginal['World_y'],c='k',label='Yolov7 Orginal trajectory')
    plt.scatter(mothod1['World_x'],mothod1['World_y'],c='blue',label='SFPF Remove Noise')
#     plt.scatter(mothod2['World_x'],mothod2['World_y'],c='green',label='Wavelet Remove Noise')
    plt.legend()
    plt.legend(loc="upper right",fontsize=12)   #设置图例字体大小

    plt.xlabel('X (m))',fontsize=16)     #设置x轴名称
    plt.legend(loc="upper right",fontsize=16)   #设置图例字体大小
    # 坐标尺
    plt.xticks(fontsize=12,rotation=0)
    plt.yticks(fontsize=12)
    # 坐标名称
    plt.rcParams.update({"font.size":16})
#     plt.title("The trajectory",fontsize=16)
    plt.ylabel('Y (m)',fontsize=16)     #设置y轴名称  
    plt.grid()
    plt.show() 
# 多维轨迹显示图
# 绘制时空分布图
def Stmap(data):
    # 计算车辆个数
    vehids = pd.unique(data["Object_id"])
    # 设置图框大小
    plt.Figure([2200,2200],dpi=1000)
    for vehid in vehids:
        # 获取第vehid辆车的轨迹
        traj_vehid = data[data.Object_id == vehid]
        # 获取x方向轨迹点
        x = traj_vehid['World_x']
        y = traj_vehid['World_y']
        v = traj_vehid['speed_xy']
        norm = matplotlib.colors.Normalize(vmin=0, vmax=50)
        # 绘制散点图
        ax = plt.scatter(x,y, marker = '.', s=1, c=v, cmap='jet_r', norm = norm)
    plt.clim(0, 25)
    plt.colorbar()
    plt.grid()
    plt.show()

## 定义滤波降噪函数

### 高斯平滑过程

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.signal
from scipy import *
import copy

def gaussian(t, fwhm):
    return np.exp(-(4*np.log(2)*t**2)/fwhm**2)

srate = 1000 #Hz
time  = np.arange(0,3,1/srate)
n     = len(time)
p     = 15 # poles for random interpolation

#noise level, measured in standard deviations
noiseamp = 5

# amplitude modular and noise level
ampl   = np.interp(np.linspace(1,p,n), np.arange(0,p), np.random.rand(p)*30)
noise  = noiseamp * np.random.randn(n)
signal = ampl + noise

fwhm    = 25 # in ms

# normalized time vector in ms
k       = 100
gtime   = 1000*np.arange(-k, k)/srate

# Gaussian window
gauswin = gaussian(gtime, fwhm)

# Compute empirical FWHM
pstPeakHalf = k + np.argmin( (gauswin[k:] -.5)**2 )
prePeakHalf = np.argmin( (gauswin -.5)**2 )

empFWHM = gtime[pstPeakHalf] - gtime[prePeakHalf]

# shouw the Gausian
# plt.plot(gtime, gauswin, 'ko-')
# plt.plot([gtime[prePeakHalf],gtime[pstPeakHalf]],[gauswin[prePeakHalf],gauswin[pstPeakHalf]],'m')
# Normalize Gaussian to unit energy
gauswin = gauswin/np.sum(gauswin)
# #title
# plt.xlabel('Time (ms)')
# plt.ylabel('Gain')
# plt.show()

# initialize filtered signal vector
filtsigG = copy.deepcopy(signal)

# implement the running mean filter
for i in range(k+1, n-k-1):
    filtsigG[i] = np.sum(signal[i-k:i+k] * gauswin)
    
plt.plot(time, signal, 'r', label='Original')
plt.plot(time, filtsigG, 'k', label='Gaussian-filtered')

plt.xlabel('Time (s)')
plt.ylabel('amp. (a.u)')
plt.legend()
plt.title('Gaussian smothing filter')
plt.show()

# filtsigMean = copy.deepcopy(signal)

# mk = 20
# for i in range(mk+1, n-mk-1):
#     filtsigMean[i] = np.mean(signal[i-mk:i+mk])
# plt.plot(time,signal,'r',label='Original')    
# plt.plot(time, filtsigMean, 'b', label='Running mean')
# plt.legend()
# plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# Generate noisy trajectory data
# n_samples = 200
# X = np.linspace(0, 10, n_samples)
# y = np.sin(X) + 0.2*np.random.randn(n_samples)

def Denoise_GP(trajectory_data):
    T = trajectory_data['frame_id'].values
    X = trajectory_data['World_x']
    y = trajectory_data['World_y']
    # Define the Gaussian Process model
    kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))

    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
    # Train the GP model
    gp.fit(T.reshape(-1, 1), y)
    gp.fit(T.reshape(-1,1),X)
    # Predict the denoised trajectory
    T_new = np.linspace(T[0], T[-1], len(T))
    y_mean, y_std = gp.predict(T_new.reshape(-1, 1), return_std=True)
    X_mean, X_std = gp.predict(T_new.reshape(-1, 1), return_std=True)
    return T, y,T_new, y_mean,y_std,X_mean, X_std

## Hampel Filter 降噪

In [ ]:
# Define the Hampel filter function
def hampel_filter(y, window_size, threshold):
    n = y.shape[0]
    y_filtered = np.zeros(n)
    for i in range(n):
        lower = max(0, i - window_size // 2)
        upper = min(n, i + window_size // 2 + 1)
        median = np.median(y[lower:upper])
        mad = np.median(np.abs(y[lower:upper] - median))
        if np.abs(y[i] - median) > threshold * mad:
            y_filtered[i] = median
        else:
            y_filtered[i] = y[i]
    return y_filtered

## 小波变换

In [ ]:
#  模块调用
# 小波变换
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import pywt 

#封装成函数
def sgn(num):
    if(num > 0.0):
        return 1.0
    elif(num == 0.0):
        return 0.0
    else:
        return -1.0
def wavelet_noising(new_df):
    data = new_df
    data = data.values.T.tolist()  # 将np.ndarray()转为列表
    w = pywt.Wavelet('sym8')#选择sym8小波基
    [ca5, cd5, cd4, cd3, cd2, cd1] = pywt.wavedec(data, w, level=5)  # 5层小波分解
    length1 = len(cd1)
    length0 = len(data)
    Cd1 = np.array(cd1)
    abs_cd1 = np.abs(Cd1)
    median_cd1 = np.median(abs_cd1)
    sigma = (1.0 / 0.6745) * median_cd1
    lamda = sigma * math.sqrt(2.0 * math.log(float(length0 ), math.e))#固定阈值计算
    usecoeffs = []
    usecoeffs.append(ca5)  # 向列表末尾添加对象
    #软硬阈值折中的方法
    a = 0.5
    for k in range(length1):
        if (abs(cd1[k]) >= lamda):
            cd1[k] = sgn(cd1[k]) * (abs(cd1[k]) - a * lamda)
        else:
            cd1[k] = 0.0
    length2 = len(cd2)
    for k in range(length2):
        if (abs(cd2[k]) >= lamda):
            cd2[k] = sgn(cd2[k]) * (abs(cd2[k]) - a * lamda)
        else:
            cd2[k] = 0.0
    length3 = len(cd3)
    for k in range(length3):
        if (abs(cd3[k]) >= lamda):
            cd3[k] = sgn(cd3[k]) * (abs(cd3[k]) - a * lamda)
        else:
            cd3[k] = 0.0
    length4 = len(cd4)
    for k in range(length4):
        if (abs(cd4[k]) >= lamda):
            cd4[k] = sgn(cd4[k]) * (abs(cd4[k]) - a * lamda)
        else:
            cd4[k] = 0.0

    length5 = len(cd5)
    for k in range(length5):
        if (abs(cd5[k]) >= lamda):
            cd5[k] = sgn(cd5[k]) * (abs(cd5[k]) - a * lamda)
        else:
            cd5[k] = 0.0
    usecoeffs.append(cd5)
    usecoeffs.append(cd4)
    usecoeffs.append(cd3)
    usecoeffs.append(cd2)
    usecoeffs.append(cd1)
    recoeffs = pywt.waverec(usecoeffs, w)#信号重构
    return recoeffs

##penalized denoising

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve

def Denoise_penalized(trajectory,alpha):

    n_samples = len(trajectory)
    n_drones = 1
    Y = np.zeros((n_samples, n_drones))

    for i in range(0,1,n_samples):
        Y[:, i] = trajectory[i]
    # Define the penalty matrix
    
    d = np.ones(n_samples)
    D = spdiags([-d, 2*d, -d], [-1, 0, 1], n_samples, n_samples)
    n_drones = 1
    # Set the regularization parameter    
    # Denoise the trajectory data with penalized denoising
    Y_denoised = np.zeros((n_samples, n_drones))
    
    for i in range(n_drones):
        Y_denoised[:, i] = spsolve(D + alpha*np.eye(n_samples), Y[:, i])
    return Y_denoised

## Laplace 变换降噪

In [ ]:
# 调所需的包
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve

# 定义拉普拉斯降噪函数，函数是单独对X或者Y进行降噪
def Laplacian_denoise(trajectory):
#     trajectory_data = trajectory_data.values
    n_samples = len(trajectory)   
    # Define the Laplacian filter
    d = np.ones(n_samples)
    D = spdiags([-d, 2*d, -d], [-1, 0, 1], n_samples, n_samples)
    # Set the regularization parameter
    alpha = 1
    # Denoise the trajectory data with Laplacian filtering
    y_denoised = spsolve(D + alpha*np.eye(n_samples), trajectory)
    return y_denoised

## 绘制对比图像

In [ ]:
def Draw_compare_figure(orginal_X,orginal_Y,filtered_X,filtered_y):
    # Plot the noisy and denoised trajectories
    plt.figure(figsize=(10, 5))
    plt.plot(orginal_X, orginal_Y, 'kx', markersize=3, label='Noisy trajectory')
    plt.plot(filtered_X,filtered_y, 'b-',markersize=0.5, label='Denoised trajectory')
    plt.legend(loc='best')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()

#执行降噪函数

#### 小波变换对轨迹进行降噪

In [ ]:
pathdata = r"/content/drive/My Drive/data_set/Mask_RCNN//Mask_orginal.csv"
YOLOV7_pathdata = r"/content/drive/My Drive/data_set/Yolodata/S_raw.csv"
mask_data = pd.read_csv(YOLOV7_pathdata)
df = pd.DataFrame(mask_data)
vehid = pd.unique(mask_data.Object_id)
Wavelet_traj = mask_data[['Object_id','frame_id','Vehicle_type']]
Wt = pd.DataFrame(Wavelet_traj)
Wt['World_x']=''
Wt['World_y']=''
Wt['speed_x']=''
Wt['speed_y']=''
Wt['acc_x']=''
Wt['acc_y']=''
Wt['Jerk_x']=''
Wt['Jerk_y']=''
# "World_y","speed_x","speed_y","acc_x","acc_y","speed_xy","acc_xy",
for id in vehid:
    B =[]
    V =[]
    ACC = []
    Jerk=[]
    traj_data = mask_data[mask_data.Object_id==id]
    Frame_id = pd.unique(traj_data.frame_id)
    TRAJ_world_x = traj_data['World_x']
    TRAJ_world_y = traj_data['World_y']
    min_row = traj_data.loc[traj_data['frame_id']== min(Frame_id),].index[0]
    max_row = traj_data.loc[traj_data['frame_id']== max(Frame_id),].index[0]
    A = df.iloc[min_row:max_row+1,[3,4]] 
    W_X = pd.array(wavelet_noising(TRAJ_world_x))
    B.append(W_X)
    W_Y = pd.array(wavelet_noising(TRAJ_world_y))
    B.append(W_Y)
    BB = pd.DataFrame(B)
    BBB = np.transpose(BB)
    if A.shape[0]<BBB.shape[0]:
        CB = BBB.iloc[0:A.shape[0],[0,1]]
    else:
        CB = BBB.iloc[0:len(BBB),[0,1]]
    Wt.iloc[min_row:max_row+1,[3,4]] = CB
    # 计算速度     
    WTraj = Wt.iloc[min_row:max_row+1,0:5]
    speed_x,speed_y = Velocity(WTraj)
    V.append(speed_x)
    V.append(speed_y)
    VV = pd.DataFrame(V)
    VVV = np.transpose(VV)
    Wt.iloc[min_row:max_row+1,[5,6]] = VVV
    # 计算加速度
    WWTraj = Wt.iloc[min_row:max_row+1,0:7]
    acc_x,acc_y = Accelection(WWTraj)
    ACC.append(acc_x)
    ACC.append(acc_y)
    ACCC = pd.DataFrame(ACC)
    AACCC = np.transpose(ACCC)
    Wt.iloc[min_row:max_row+1,[7,8]] = AACCC
    # 计算加加速度
Wt.to_csv('/content/drive/My Drive/data_set/Wavelet_denoise/Yolov7_denoise.csv')

#### Hampel Filter 降噪

In [ ]:
pathdata = r"/content/drive/My Drive/data_set/Mask_RCNN//Mask_orginal.csv"
YOLOV7_pathdata = r"/content/drive/My Drive/data_set/Yolodata/S_raw.csv"
mask_data = pd.read_csv(YOLOV7_pathdata)
df = pd.DataFrame(mask_data)
vehid = pd.unique(mask_data.Object_id)
Wavelet_traj = mask_data[['Object_id','frame_id','Vehicle_type']]
Wt = pd.DataFrame(Wavelet_traj)
Wt['World_x']=''
Wt['World_y']=''
Wt['speed_x']=''
Wt['speed_y']=''
Wt['acc_x']=''
Wt['acc_y']=''
Wt['Jerk_x']=''
Wt['Jerk_y']=''
# "World_y","speed_x","speed_y","acc_x","acc_y","speed_xy","acc_xy",
for id in vehid:
    B =[]
    V =[]
    ACC = []
    Jerk=[]
    traj_data = mask_data[mask_data.Object_id==id]
    Frame_id = pd.unique(traj_data.frame_id)
    TRAJ_world_x = traj_data['World_x'].values
    TRAJ_world_y = traj_data['World_y'].values
    min_row = traj_data.loc[traj_data['frame_id']== min(Frame_id),].index[0]
    max_row = traj_data.loc[traj_data['frame_id']== max(Frame_id),].index[0]
    A = df.iloc[min_row:max_row+1,[3,4]] 
    W_X = pd.array(hampel_filter(TRAJ_world_x,10,3))
    B.append(W_X)
    W_Y = pd.array(hampel_filter(TRAJ_world_y,10,3))
    B.append(W_Y)
    BB = pd.DataFrame(B)
    BBB = np.transpose(BB)
    if A.shape[0]<BBB.shape[0]:
        CB = BBB.iloc[0:A.shape[0],[0,1]]
    else:
        CB = BBB.iloc[0:len(BBB),[0,1]]
    Wt.iloc[min_row:max_row+1,[3,4]] = CB
    # 计算速度     
    WTraj = Wt.iloc[min_row:max_row+1,0:5]
    speed_x,speed_y = Velocity(WTraj)
    V.append(speed_x)
    V.append(speed_y)
    VV = pd.DataFrame(V)
    VVV = np.transpose(VV)
    Wt.iloc[min_row:max_row+1,[5,6]] = VVV
    # 计算加速度
    WWTraj = Wt.iloc[min_row:max_row+1,0:7]
    acc_x,acc_y = Accelection(WWTraj)
    ACC.append(acc_x)
    ACC.append(acc_y)
    ACCC = pd.DataFrame(ACC)
    AACCC = np.transpose(ACCC)
    Wt.iloc[min_row:max_row+1,[7,8]] = AACCC
    # 计算加加速度
Wt.to_csv('/content/drive/My Drive/data_set/Hampel_denoise/Yolov7_denoise.csv')

#### Penalized 轨迹降噪

In [ ]:
pathdata = r"/content/drive/My Drive/data_set/Mask_RCNN//Mask_orginal.csv"
YOLOV7_pathdata = r"/content/drive/My Drive/data_set/Yolodata/S_raw.csv"
mask_data = pd.read_csv(YOLOV7_pathdata)
df = pd.DataFrame(mask_data)
vehid = pd.unique(mask_data.Object_id)
Wavelet_traj = mask_data[['Object_id','frame_id','Vehicle_type']]
Wt = pd.DataFrame(Wavelet_traj)
Wt['World_x']=''
Wt['World_y']=''
Wt['speed_x']=''
Wt['speed_y']=''
Wt['acc_x']=''
Wt['acc_y']=''
Wt['Jerk_x']=''
Wt['Jerk_y']=''
# "World_y","speed_x","speed_y","acc_x","acc_y","speed_xy","acc_xy",
for id in vehid:
    B =[]
    V =[]
    ACC = []
    Jerk=[]
    traj_data = mask_data[mask_data.Object_id==id]
    Frame_id = pd.unique(traj_data.frame_id)
    TRAJ_world_x = traj_data['World_x'].values
    TRAJ_world_y = traj_data['World_y'].values
    min_row = traj_data.loc[traj_data['frame_id']== min(Frame_id),].index[0]
    max_row = traj_data.loc[traj_data['frame_id']== max(Frame_id),].index[0]
    A = df.iloc[min_row:max_row+1,[3,4]] 
    W_X = pd.array(Denoise_penalized(TRAJ_world_x,1))
    B.append(W_X)
    W_Y = pd.array(Denoise_penalized(TRAJ_world_y,1))
    B.append(W_Y)
    BB = pd.DataFrame(B)
    BBB = np.transpose(BB)
    if A.shape[0]<BBB.shape[0]:
        CB = BBB.iloc[0:A.shape[0],[0,1]]
    else:
        CB = BBB.iloc[0:len(BBB),[0,1]]
    Wt.iloc[min_row:max_row+1,[3,4]] = CB
    # 计算速度     
    WTraj = Wt.iloc[min_row:max_row+1,0:5]
    speed_x,speed_y = Velocity(WTraj)
    V.append(speed_x)
    V.append(speed_y)
    VV = pd.DataFrame(V)
    VVV = np.transpose(VV)
    Wt.iloc[min_row:max_row+1,[5,6]] = VVV
    # 计算加速度
    WWTraj = Wt.iloc[min_row:max_row+1,0:7]
    acc_x,acc_y = Accelection(WWTraj)
    ACC.append(acc_x)
    ACC.append(acc_y)
    ACCC = pd.DataFrame(ACC)
    AACCC = np.transpose(ACCC)
    Wt.iloc[min_row:max_row+1,[7,8]] = AACCC
    # 计算加加速度
Wt.to_csv('/content/drive/My Drive/data_set/Penalized_denoise/Yolov7_denoise.csv')

#### 利用Laplace Filter对轨迹进行降噪处理

In [ ]:
pathdata = r"/content/drive/My Drive/data_set/Mask_RCNN//Mask_orginal.csv"
YOLOV7_pathdata = r"/content/drive/My Drive/data_set/Yolodata/S_raw.csv"
mask_data = pd.read_csv(YOLOV7_pathdata)
df = pd.DataFrame(mask_data)
vehid = pd.unique(mask_data.Object_id)
Wavelet_traj = mask_data[['Object_id','frame_id','Vehicle_type']]
Wt = pd.DataFrame(Wavelet_traj)
Wt['World_x']=''
Wt['World_y']=''
Wt['speed_x']=''
Wt['speed_y']=''
Wt['acc_x']=''
Wt['acc_y']=''
Wt['Jerk_x']=''
Wt['Jerk_y']=''
# "World_y","speed_x","speed_y","acc_x","acc_y","speed_xy","acc_xy",
for id in vehid:
    B =[]
    V =[]
    ACC = []
    Jerk=[]
    traj_data = mask_data[mask_data.Object_id==id]
    Frame_id = pd.unique(traj_data.frame_id)
    TRAJ_world_x = traj_data['World_x'].values
    TRAJ_world_y = traj_data['World_y'].values
    min_row = traj_data.loc[traj_data['frame_id']== min(Frame_id),].index[0]
    max_row = traj_data.loc[traj_data['frame_id']== max(Frame_id),].index[0]
    A = df.iloc[min_row:max_row+1,[3,4]] 

    W_X = pd.array(Laplacian_denoise(TRAJ_world_x))
    B.append(W_X)
    W_Y = pd.array(Laplacian_denoise(TRAJ_world_y))
    B.append(W_Y)
    BB = pd.DataFrame(B)
    BBB = np.transpose(BB)
    if A.shape[0]<BBB.shape[0]:
        CB = BBB.iloc[0:A.shape[0],[0,1]]
    else:
        CB = BBB.iloc[0:len(BBB),[0,1]]
    Wt.iloc[min_row:max_row+1,[3,4]] = CB
    # 计算速度     
    WTraj = Wt.iloc[min_row:max_row+1,0:5]
    speed_x,speed_y = Velocity(WTraj)
    V.append(speed_x)
    V.append(speed_y)
    VV = pd.DataFrame(V)
    VVV = np.transpose(VV)
    Wt.iloc[min_row:max_row+1,[5,6]] = VVV
    # 计算加速度
    WWTraj = Wt.iloc[min_row:max_row+1,0:7]
    acc_x,acc_y = Accelection(WWTraj)
    ACC.append(acc_x)
    ACC.append(acc_y)
    ACCC = pd.DataFrame(ACC)
    AACCC = np.transpose(ACCC)
    Wt.iloc[min_row:max_row+1,[7,8]] = AACCC
    # 计算加加速度
Wt.to_csv('/content/drive/My Drive/data_set/Laplace_denoise/Yolov7_denoise.csv')

### LSTM_DENOISE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Reshape

# Generate noisy vehicle trajectory data
def LSTM_denoise(trajectory):
    n_samples = len(trajectory)
    y = trajectory
    t = 20
    # Create input/output pairs for training the LSTM
    window_size = t
    X_train = np.zeros((n_samples-window_size, window_size, 1))
    y_train = np.zeros((n_samples-window_size, 1))
    for i in range(n_samples-window_size):
        X_train[i, :, 0] = y[i:i+window_size]
        y_train[i, 0] = y[i+window_size]

    # Create an LSTM model for denoising the trajectory data
    model = Sequential([
        LSTM(32, input_shape=(window_size, 1), return_sequences=True),
        LSTM(16, return_sequences=True),
        LSTM(8),
        Dense(1)
    ])

    # Compile the model and train it on the noisy trajectory data
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=1000)

    # Use the trained model to denoise the trajectory data
    y_denoised = model.predict(X_train)

    list_a_W_X = [item for sublist in y_denoised for item in sublist]
    list_A_W_X = list(list_a_W_X)
    list_B_W_X = list(trajectory[0:t])
    list_B_W_X.extend(list_A_W_X)
    Lstm_denoise=np.array(list_B_W_X)
    return Lstm_denoise

In [ ]:
pathdata = r"/content/drive/My Drive/data_set/Mask_RCNN//Mask_orginal.csv"
YOLOV7_pathdata = r"/content/drive/My Drive/data_set/Yolodata/S_raw.csv"
mask_data = pd.read_csv(YOLOV7_pathdata)
df = pd.DataFrame(mask_data)
vehid = pd.unique(mask_data.Object_id)
Wavelet_traj = mask_data[['Object_id','frame_id','Vehicle_type']]
Wt = pd.DataFrame(Wavelet_traj)
Wt['World_x']=''
Wt['World_y']=''
Wt['speed_x']=''
Wt['speed_y']=''
Wt['acc_x']=''
Wt['acc_y']=''
Wt['Jerk_x']=''
Wt['Jerk_y']=''
# "World_y","speed_x","speed_y","acc_x","acc_y","speed_xy","acc_xy",
for id in vehid:
    print('----------------------------------------------------------')
    B =[]
    V =[]
    ACC = []
    Jerk=[]
    traj_data = mask_data[mask_data.Object_id==id]
    Frame_id = pd.unique(traj_data.frame_id)
    TRAJ_world_x = traj_data['World_x'].values
    TRAJ_world_y = traj_data['World_y'].values
    min_row = traj_data.loc[traj_data['frame_id']== min(Frame_id),].index[0]
    max_row = traj_data.loc[traj_data['frame_id']== max(Frame_id),].index[0]
    A = df.iloc[min_row:max_row+1,[3,4]] 
    W_X = LSTM_denoise(TRAJ_world_x)
    B.append(W_X)
    W_Y = LSTM_denoise(TRAJ_world_y)
    B.append(W_Y)
    BB = pd.DataFrame(B)
    BBB = np.transpose(BB)
    if A.shape[0]<BBB.shape[0]:
        CB = BBB.iloc[0:A.shape[0],[0,1]]
    else:
        CB = BBB.iloc[0:len(BBB),[0,1]]
    Wt.iloc[min_row:max_row+1,[3,4]] = CB
    # 计算速度     
    WTraj = Wt.iloc[min_row:max_row+1,0:5]
    speed_x,speed_y = Velocity(WTraj)
    V.append(speed_x)
    V.append(speed_y)
    VV = pd.DataFrame(V)
    VVV = np.transpose(VV)
    Wt.iloc[min_row:max_row+1,[5,6]] = VVV
    # 计算加速度
    WWTraj = Wt.iloc[min_row:max_row+1,0:7]
    acc_x,acc_y = Accelection(WWTraj)
    ACC.append(acc_x)
    ACC.append(acc_y)
    ACCC = pd.DataFrame(ACC)
    AACCC = np.transpose(ACCC)
    Wt.iloc[min_row:max_row+1,[7,8]] = AACCC
    # 计算加加速度
Wt.to_csv('/content/drive/My Drive/data_set/LSTM_denoise/Yolov7_denoise.csv')

### 卷积神经网络函数定义，CNN_denoise

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Reshape

def CNN_denoise(trajectory):
    # Generate noisy vehicle trajectory data
    n_samples = len(trajectory)
    print(n_samples)
    y = trajectory
    # Create input/output pairs for training the CNN
    window_size = 25
    X_train = np.zeros((n_samples-window_size, window_size, 1))
    y_train = np.zeros((n_samples-window_size, 1))
    for i in range(n_samples-window_size):
        X_train[i, :, 0] = y[i:i+window_size]
        y_train[i, 0] = y[i+window_size]
    # Create a CNN model for denoising the trajectory data
    model = Sequential([
        Conv1D(16, 3, activation='relu', input_shape=(window_size, 1)),
        MaxPooling1D(2),
        Conv1D(32, 3, activation='relu'),
        MaxPooling1D(2),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    # Compile the model and train it on the noisy trajectory data
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100)
    # Use the trained model to denoise the trajectory data
    y_denoised = model.predict(X_train)
    list_a_W_X = [item for sublist in y_denoised for item in sublist]
    list_A_W_X = list(list_a_W_X)
    list_B_W_X = list(trajectory[0:25])
    list_B_W_X.extend(list_A_W_X)
    a_denoise=np.array(list_B_W_X) 
    return a_denoise

###卷积神经网络降噪-CNN_denoise

In [ ]:
pathdata = r"/content/drive/My Drive/data_set/Mask_RCNN//Mask_orginal.csv"
YOLOV7_pathdata = r"/content/drive/My Drive/data_set/Yolodata/S_raw.csv"
mask_data = pd.read_csv(YOLOV7_pathdata)
df = pd.DataFrame(mask_data)
vehid = pd.unique(mask_data.Object_id)
Wavelet_traj = mask_data[['Object_id','frame_id','Vehicle_type']]
Wt = pd.DataFrame(Wavelet_traj)
Wt['World_x']=''
Wt['World_y']=''
Wt['speed_x']=''
Wt['speed_y']=''
Wt['acc_x']=''
Wt['acc_y']=''
Wt['Jerk_x']=''
Wt['Jerk_y']=''
# "World_y","speed_x","speed_y","acc_x","acc_y","speed_xy","acc_xy",
for id in vehid:
    print('----------------------------------------------------------')
    B =[]
    V =[]
    ACC = []
    Jerk=[]
    traj_data = mask_data[mask_data.Object_id==id]
    Frame_id = pd.unique(traj_data.frame_id)
    TRAJ_world_x = traj_data['World_x'].values
    TRAJ_world_y = traj_data['World_y'].values
    min_row = traj_data.loc[traj_data['frame_id']== min(Frame_id),].index[0]
    max_row = traj_data.loc[traj_data['frame_id']== max(Frame_id),].index[0]
    A = df.iloc[min_row:max_row+1,[3,4]] 
    W_X = CNN_denoise(TRAJ_world_x)
    B.append(W_X)
    W_Y = CNN_denoise(TRAJ_world_y)
    B.append(W_Y)
    BB = pd.DataFrame(B)
    BBB = np.transpose(BB)
    if A.shape[0]<BBB.shape[0]:
        CB = BBB.iloc[0:A.shape[0],[0,1]]
    else:
        CB = BBB.iloc[0:len(BBB),[0,1]]
    Wt.iloc[min_row:max_row+1,[3,4]] = CB
    # 计算速度     
    WTraj = Wt.iloc[min_row:max_row+1,0:5]
    speed_x,speed_y = Velocity(WTraj)
    V.append(speed_x)
    V.append(speed_y)
    VV = pd.DataFrame(V)
    VVV = np.transpose(VV)
    Wt.iloc[min_row:max_row+1,[5,6]] = VVV
    # 计算加速度
    WWTraj = Wt.iloc[min_row:max_row+1,0:7]
    acc_x,acc_y = Accelection(WWTraj)
    ACC.append(acc_x)
    ACC.append(acc_y)
    ACCC = pd.DataFrame(ACC)
    AACCC = np.transpose(ACCC)
    Wt.iloc[min_row:max_row+1,[7,8]] = AACCC
    # 计算加加速度
Wt.to_csv('/content/drive/My Drive/data_set/CNN_denoise/Yolov7_denoise.csv')

### 基于自编码生成模型的降噪

#### 基于自编码处理轨迹数据集

### 基于生成模型的轨迹降噪函数定义

In [ ]:
# 列出你云盘里有啥
!ls "/content/drive/My Drive/data_set/Ground_truth"

基于自编码轨迹数据处理

### 对比不同处理结果，计算对比指标

In [ ]:
## 导入对应id数据
# 导入高低密度下的车辆对应id列表
# 导入ground turth dataset
pathG_Y_ID = r'/content/drive/My Drive/data_set/ID_MATCH/YOLOV7_GROUND/high.csv'
G_Y_ID_high = pd.read_csv(pathG_Y_ID)
gound_yolo_ID_high = pd.DataFrame(G_Y_ID_high) #swatch the dict to dataframe
# 导入低密度下的车辆对应id列表
# 导入ground turth dataset
pathG_Y_ID = r'/content/drive/My Drive/data_set/ID_MATCH/YOLOV7_GROUND/low.csv'
G_Y_ID_low = pd.read_csv(pathG_Y_ID)
gound_yolo_ID_low = pd.DataFrame(G_Y_ID_low) #swatch the dict to dataframe

## 导入ground_truth and Mask_RCNN对应id数据
# 导入低密度下的车辆对应id的列表
pathG_M_ID = r'/content/drive/My Drive/data_set/ID_MATCH/Mask_Ground/low.csv'
G_M_ID_LOW = pd.read_csv(pathG_M_ID)
gound_mask_ID_LOW = pd.DataFrame(G_M_ID_LOW) #swatch the dict to dataframe
# 导入高密度下的车辆对应id列表
# 导入ground turth dataset
pathG_m_HID = r'/content/drive/My Drive/data_set/ID_MATCH/Mask_Ground/high.csv'
G_M_ID_HIHG = pd.read_csv(pathG_m_HID)
gound_mask_ID_high = pd.DataFrame(G_M_ID_HIHG) #swatch the dict to dataframe

Ground_truth_traj = r'/content/drive/My Drive/data_set/Ground_truth/Geroge_ST_trajectory.csv'
Ground_truth = pd.read_csv(Ground_truth_traj)
G_TRAJ_T = pd.DataFrame(Ground_truth)

### Define the indicator 

In [ ]:
# 对齐轨迹点的函数
def Alignment(Groundtruthebike,Comparetraj):
#     traj_ground0 = Groundtruthebike[Groundtruthebike.Object_id ==ground_id]
#     print(Groundtruthebike,Comparetraj)
    min_Y_G = min(pd.array(Groundtruthebike.World_y))
    max_Y_G = max(pd.array(Groundtruthebike.World_y))
    # CV输出的轨迹数据
#     Comparetraj0 = Comparetraj[Comparetraj.Object_id == yolo_id]
    min_Y_C = min(pd.array(Comparetraj.World_y))
    max_Y_C = max(pd.array(Comparetraj.World_y))
    bottom = max(min_Y_G,min_Y_C)
    top = min(max_Y_G,max_Y_C)
    #让轨迹点对齐
    # 需要对齐时间帧
#     print("------------------------1--------------------------")
    traj_ground1 = Groundtruthebike[Groundtruthebike.World_y>=bottom]
    traj_ground = traj_ground1[traj_ground1.World_y<=top] 
#     traj_ground['G_frameid'] = range(1:traj_ground.shape[0])
    Comparetraj1 = Comparetraj[Comparetraj.World_y>=bottom]
    traj_compare = Comparetraj1[Comparetraj1.World_y<=top] 
#     print("-----------------------2---------------------------")
    #设置时间帧对齐的方法
    if traj_ground.shape[0]>=traj_compare.shape[0]:
        frame_ids =  traj_ground.frame_id[0:traj_compare.shape[0]]
#         print("----------------------------1------------------------")
        traj_compare['G_frameid'] = pd.array(frame_ids)
#         print(traj_compare.shape[0])        
    else:
        frame_ids = traj_ground.frame_id
#         print("----------------------------2------------------------")
#         print(traj_ground.shape[0])
        traj_compare = traj_compare[0:traj_ground.shape[0]]
#         print(traj_compare.shape[0])
        traj_compare['G_frameid'] = pd.array(frame_ids)   
#         print(traj_compare)     
    return traj_ground,traj_compare

In [ ]:
# 计算轨迹中的速度/加速度/加加速度
def XY(groundtraj,caompartraj):
    g_World_x = np.array(groundtraj['World_x'].astype(float))
    g_World_y = np.array(groundtraj['World_y'].astype(float))
    com_World_x = np.array(caompartraj['World_x'].astype(float))
    com_World_y = np.array(caompartraj['World_y'].astype(float))    
    return g_World_x,g_World_y,com_World_x,com_World_y

def Velocity(trajdata):
    """recalculate the velocity, the velocity contain the X_velocity and Y-velocity and the speed"""
    """定义初速度为0"""
    len_x = trajdata.shape[0]
    wordld_x = np.array(trajdata.World_x)
    wordld_y = np.array(trajdata.World_y)
    velocity_x = (wordld_x[1:len_x]-wordld_x[0:len_x-1])/0.04
    velocity_y = (wordld_y[1:len_x]-wordld_y[0:len_x-1])/0.04
    velocity_x = np.insert(velocity_x,0,0)
    velocity_y = np.insert(velocity_y,0,0)
    return velocity_x,velocity_y

def Accelection(trajdata):
    """定结束时刻的加速度为0"""
    len_x = trajdata.shape[0]
    velocity_x = np.array(trajdata.speed_x)
    velocity_y = np.array(trajdata.speed_y)
    accelection_x = (velocity_x[1:len_x]-velocity_x[0:len_x-1])/0.04
    accelection_y = (velocity_y[1:len_x]-velocity_y[0:len_x-1])/0.04
    accelection_x = np.insert(accelection_x,0,0)
    accelection_y = np.insert(accelection_y,0,0)

In [ ]:
def Samelen(GTRAJ,COMTRAJ): 
    frame_len = min(len(GTRAJ),len(COMTRAJ))
    traj_ground2 = GTRAJ.iloc[:frame_len]
    traj_compare2 = COMTRAJ[:frame_len]
    return traj_ground2,traj_compare2

In [ ]:
# 二维坐标显示
def Trajmap(traj_ground,traj_compare):
    # Trajdatas对比轨迹，traj_compare是真实轨迹
    fig1 = plt.figure(figsize=(8,4))   
    g_World_x,g_World_y,com_World_x,com_World_y = XY(traj_ground,traj_compare) 
    plt.plot(g_World_x,g_World_y,color="blue",label='Ground trajectory')
    plt.plot(com_World_x,com_World_y,color="red",label='Yolov7 trajectory')
    plt.legend()
    plt.legend(loc="upper right",fontsize=12)   #设置图例字体大小
    plt.xlabel('X(m))',fontsize=16)     #设置x轴名称
    plt.legend(loc="upper right",fontsize=16)   #设置图例字体大小
    # 坐标尺
    plt.xticks(fontsize=12,rotation=0)
    plt.yticks(fontsize=12)
    # 坐标名称
    plt.rcParams.update({"font.size":16})
    plt.title("The trajectory",fontsize=16)
    plt.ylabel('Y(m)',fontsize=16)     #设置y轴名称
    plt.grid()
    plt.show()
    return fig1  

In [ ]:
# RMSE
def Index_rmse(traj_ground,traj_compare):
    #比较真实轨迹与CV输出轨迹以及算法输出轨迹之间的差异
    #选择相同的起点坐标，后面选取相同长度的轨迹点，比较两个数据集长度
#     print(traj_ground,traj_compare)
    ground_truth,Comparetraj = Samelen(traj_ground,traj_compare)
    len_ground_truth = ground_truth.shape[0]
    len_Compare_traj = Comparetraj.shape[0]     
    num_traj_id = min(len_ground_truth,len_Compare_traj)    
    Ground_traj = ground_truth[0:num_traj_id]    
    Compar_traj = Comparetraj[0:num_traj_id]    
    Ground_traj_world_x = np.array(Ground_traj.World_x)
    Ground_traj_world_y = np.array(Ground_traj.World_y)
    Compar_traj_world_x = np.array(Compar_traj.World_x)
    Compar_traj_world_y = np.array(Compar_traj.World_y)
    #计算真实轨迹点的差异
    deltx = np.array(Ground_traj_world_x - Compar_traj_world_x)
    delty = np.array(Ground_traj_world_y - Compar_traj_world_y)
    RMSE = math.sqrt((np.sum(deltx*deltx) + np.sum(deltx*deltx))/num_traj_id)
    return RMSE

In [ ]:
# 计算信噪比
def SNR(traj_ground,traj_compare):
    # 计算信号与噪声的比值
    #比较真实轨迹与CV输出轨迹以及算法输出轨迹之间的差异
    #选择相同的起点坐标，后面选取相同长度的轨迹点，比较两个数据集长度
    # print(traj_ground,traj_compare)
    ground_truth,Comparetraj = Samelen(traj_ground,traj_compare)
    len_ground_truth = ground_truth.shape[0]
    len_Compare_traj = Comparetraj.shape[0]     
    num_traj_id = min(len_ground_truth,len_Compare_traj)    
    Ground_traj = ground_truth[0:num_traj_id]    
    Compar_traj = Comparetraj[0:num_traj_id]    
    Ground_traj_world_x = np.array(Ground_traj.World_x)
    Ground_traj_world_y = np.array(Ground_traj.World_y)
    Compar_traj_world_x = np.array(Compar_traj.World_x)
    Compar_traj_world_y = np.array(Compar_traj.World_y)
    #计算轨迹噪声    
    deltx = np.array(Ground_traj_world_x - Compar_traj_world_x)
    delty = np.array(Ground_traj_world_y - Compar_traj_world_y)
    Noise = (np.sum(deltx*deltx) + np.sum(deltx*deltx))
    # 计算轨迹信号
    SIGNAL = np.sum(Compar_traj_world_x*Compar_traj_world_x+Compar_traj_world_y*Compar_traj_world_y)
    SNRR = 10*math.log(SIGNAL/Noise)
    return SNRR

In [ ]:
# 计算R方
def RR(traj_ground,traj_compare):
    #计算轨迹的相似性
    #比较真实轨迹与CV输出轨迹以及算法输出轨迹之间的差异
    #选择相同的起点坐标，后面选取相同长度的轨迹点，比较两个数据集长度
    # print(traj_ground,traj_compare)
    ground_truth,Comparetraj = Samelen(traj_ground,traj_compare)
    len_ground_truth = ground_truth.shape[0]
    len_Compare_traj = Comparetraj.shape[0]     
    num_traj_id = min(len_ground_truth,len_Compare_traj)    
    Ground_traj = ground_truth[0:num_traj_id]    
    Compar_traj = Comparetraj[0:num_traj_id]    
    Ground_traj_world_x = np.array(Ground_traj.World_x)
    Ground_traj_world_y = np.array(Ground_traj.World_y)
    Compar_traj_world_x = np.array(Compar_traj.World_x)
    Compar_traj_world_y = np.array(Compar_traj.World_y)
    
    #将坐标转换为距离起点的距离
    DIS_G =  pd.DataFrame(Ground_traj_world_x*Ground_traj_world_x+Ground_traj_world_y*Ground_traj_world_y)
    Dis_Ground_traj_world = DIS_G.apply(np.sqrt)
    DIS_C =  pd.DataFrame(Compar_traj_world_x*Compar_traj_world_x+Compar_traj_world_y*Compar_traj_world_y)
    Dis_Compar_traj_world = DIS_C.apply(np.sqrt)
    
    M_DIS_G = np.mean(Dis_Ground_traj_world)
    M_DIS_C = np.mean(Dis_Compar_traj_world)
    G_delta = Dis_Ground_traj_world-M_DIS_G
    C_delta = Dis_Compar_traj_world- M_DIS_C
    up_1 = np.array(G_delta)
    up_2 = np.array(C_delta)
    UP_total = np.multiply(up_1,up_2)
    UP_total_value = np.sum(UP_total)
    
    
    Botm_1 = np.sum(G_delta*G_delta)
#     Botm_2 = np.sum(C_delta*C_delta)
    Botm_2 = 1
    Botm_total_value = np.multiply(Botm_1,Botm_2)
    
    print("-----------------")
    Rr = (UP_total_value/Botm_total_value)
    print(Rr)
    return Rr   

In [ ]:
import numpy as np
from astropy.units import Ybarn
import math
 
def computeCorrelation(X, Y):
    xBar = np.mean(X)
    yBar = np.mean(Y)
    SSR = 0
    varX = 0
    varY = 0
    for i in range(0 , len(X)):
        diffXXBar = X[i] - xBar
        diffYYBar = Y[i] - yBar
        SSR += (diffXXBar * diffYYBar)
        varX +=  diffXXBar**2
        varY += diffYYBar**2
    
    SST = math.sqrt(varX * varY)
    if SST==0:
        SST =1
    return SSR / SST
 
def RRR(traj_ground,traj_compare): 
    ground_truth,Comparetraj = Samelen(traj_ground,traj_compare)
    len_ground_truth = ground_truth.shape[0]
    len_Compare_traj = Comparetraj.shape[0]     
    num_traj_id = min(len_ground_truth,len_Compare_traj)    
    Ground_traj = ground_truth[0:num_traj_id]    
    Compar_traj = Comparetraj[0:num_traj_id]    
    Ground_traj_world_x = np.array(Ground_traj.World_x)
    Ground_traj_world_y = np.array(Ground_traj.World_y)
    Compar_traj_world_x = np.array(Compar_traj.World_x)
    Compar_traj_world_y = np.array(Compar_traj.World_y)
    X_RR = computeCorrelation(Ground_traj_world_x, Compar_traj_world_x)
    Y_RR = computeCorrelation(Ground_traj_world_y, Compar_traj_world_y)
    return  X_RR,Y_RR

In [ ]:
from scipy.spatial.distance import directed_hausdorff
def Hausdorff(traj_ground,traj_compare):
    ground_truth,Comparetraj = Samelen(traj_ground,traj_compare)
    len_ground_truth = ground_truth.shape[0]
    len_Compare_traj = Comparetraj.shape[0]     
    num_traj_id = min(len_ground_truth,len_Compare_traj)    
    Ground_traj = ground_truth[0:num_traj_id]    
    Compar_traj = Comparetraj[0:num_traj_id]    
    Ground_traj_world_XY = Ground_traj[['World_x','World_y']]
    Compar_traj_world_xy = Compar_traj[['World_x','World_y']]
    Dir_hausdorff_1 = directed_hausdorff(Ground_traj_world_XY, Compar_traj_world_xy)[0]
    Dir_hausdorff_2 = directed_hausdorff(Compar_traj_world_xy, Ground_traj_world_XY)[0]
    Dir_hausdorff_3 = directed_hausdorff(Compar_traj_world_xy, Ground_traj_world_XY)[1:]
    Hausdorff_dis =  max(directed_hausdorff(Ground_traj_world_XY, Compar_traj_world_xy)[0], directed_hausdorff(Compar_traj_world_xy, Ground_traj_world_XY)[0])
    return Hausdorff_dis

In [ ]:
# 列出你云盘里有啥
!ls "/content/drive/My Drive/data_set/Yolodata"

In [ ]:
Ebike_traj_g = G_TRAJ_T[G_TRAJ_T['Vehicle_type']==22]

In [ ]:
# gound_yolo_ID_high

### 对比轨迹

In [ ]:
# ground truth
Groundtruthebike = Ebike_traj_g

# Yolov7 轨迹数据
yolov7data = r'/content/drive/My Drive/data_set/Yolodata/S_raw.csv'
Yolov7_traj_orginal = pd.read_csv(yolov7data)
Yolov7_traj_orginal = pd.DataFrame(Yolov7_traj_orginal)
Yolov7_org = Yolov7_traj_orginal[Yolov7_traj_orginal.Vehicle_type==22]


# 导入轨迹对应轨迹的ID匹配表
for id in range(len(gound_yolo_ID_high)):
    ground_id = gound_yolo_ID_high.ground_ID[id]
    yolo_id = gound_yolo_ID_high.yolo_ID[id]
    # 真实轨迹数据 
    traj_ground1 = Groundtruthebike[Groundtruthebike.Object_id ==ground_id]
    traj_ground0 = pd.DataFrame(traj_ground1)
    
    # CV输出的轨迹数据
    traj_compare1 = Yolov7_org[Yolov7_org.Object_id == yolo_id]
    traj_compare0 = pd.DataFrame(traj_compare1)

    if traj_compare0.shape[0]>=10:

        traj_ground,traj_compare = Alignment(traj_ground0,traj_compare0)

        #  显示二维平面轨迹
        fig1 = Trajmap(traj_ground,traj_compare)  
#         SNRRR = SNR(traj_ground,traj_compare)
#         print(SNRRR)
#         R = RR(traj_ground,traj_compare)
#         X_RR,Y_RR = RRR(traj_ground,traj_compare)
#         print(X_RR,Y_RR)
#         Hausdorffdis = Hausdorff(traj_ground,traj_compare)
#         print(Hausdorffdis)
        #  显示时间速度对比图像   
#         fig2 = Timespeed(traj_ground,traj_compare) 
        #   显示时间加速度对比图像
#         fig3 = Timeacc(traj_ground,traj_compare) 
        #  显示时间加加速度对比图像
#         fig4 = Timejerk(traj_ground,traj_compare)

#### 计算轨迹之间的差异指标
##### 将轨迹数据分为无交互、弱交互和强交互三个阶段，分别比较不同类型轨迹的RMSE
##### 第二步，对比cv输出轨迹与真实轨迹之间的差异，选取强交互情景下的轨迹做对比分析：
##### 第三部分：对比降噪算法应用之后的轨迹
##### 第一步，对比整体参数，
##### 第二步，计算微观RMSE、DTWD等检验指标

In [ ]:
# 列出你云盘里有啥
!ls "/content/drive/My Drive/data_set/result_compare/"

In [ ]:
## 导入对应id数据
# 导入高低密度下的车辆对应id列表
# 导入ground turth dataset
pathG_Y_ID = r'/content/drive/My Drive/data_set/ID_MATCH/YOLOV7_GROUND/high.csv'
G_Y_ID_high = pd.read_csv(pathG_Y_ID)
gound_yolo_ID_high = pd.DataFrame(G_Y_ID_high) #swatch the dict to dataframe
# print(gound_yolo_ID_high)
# 导入低密度下的车辆对应id列表
# 导入ground turth dataset
pathG_Y_ID = r'/content/drive/My Drive/data_set/ID_MATCH/YOLOV7_GROUND/low.csv'
G_Y_ID_low = pd.read_csv(pathG_Y_ID)
gound_yolo_ID_low = pd.DataFrame(G_Y_ID_low) #swatch the dict to dataframe
# print(gound_yolo_ID_low)
## 导入ground_truth and Mask_RCNN对应id数据
# 导入低密度下的车辆对应id的列表
pathG_M_ID = r'/content/drive/My Drive/data_set/ID_MATCH/Mask_Ground/low.csv'
G_M_ID_LOW = pd.read_csv(pathG_M_ID)
gound_mask_ID_LOW = pd.DataFrame(G_M_ID_LOW) #swatch the dict to dataframe
# 导入高密度下的车辆对应id列表
# 导入ground turth dataset
pathG_m_HID = r'/content/drive/My Drive/data_set/ID_MATCH/Mask_Ground/high.csv'
G_M_ID_HIHG = pd.read_csv(pathG_m_HID)
gound_mask_ID_high = pd.DataFrame(G_M_ID_HIHG) #swatch the dict to dataframe

# 进行敏感性分析
marginal_Y_ID = r'/content/drive/My Drive/data_set/ID_MATCH/YOLOV7_GROUND/all.csv'
marginalmx = pd.read_csv(marginal_Y_ID)
marginalmx_ID_ALL = pd.DataFrame(marginalmx) #swatch the dict to dataframe

### 计算差异指标的表格
#### 是否可以将所有的表格汇集到一个表格中得到最后的表格呢？

In [ ]:
# compar_data
compar_data = r'/content/drive/My Drive/data_set/Yolodata/S_raw.csv'


compar_traj= pd.read_csv(compar_data)
compar_traj = pd.DataFrame(compar_traj)
compar_traj_ebike = compar_traj[compar_traj.Vehicle_type==22]
# 计算多个loop对应轨迹的RMSE

gound_yolo_ID_high = gound_yolo_ID_low
hang = gound_yolo_ID_high.shape[0]
LoopRMSE = np.zeros((hang,4))
LoopSNR = np.zeros((hang,4))
LoopHausdorff = np.zeros((hang,4))
LoopRRsquared_x = np.zeros((hang,4))
LoopRRsquared_y = np.zeros((hang,4))

RMSE_allloop = pd.DataFrame(LoopRMSE)
SNR_allloop = pd.DataFrame(LoopSNR)
Hausdorff_allloop = pd.DataFrame(LoopHausdorff)
Rsquared_allloop_x = pd.DataFrame(LoopRRsquared_x)
Rsquared_allloop_y = pd.DataFrame(LoopRRsquared_y)

for loop in range(0,1):
    Rmse_list =[]
    SNR_list = []
    Hausdorff_list = []
    Rsquared_x_list = []
    Rsquared_y_list = []
    for id in range(len(gound_yolo_ID_high)):      
        ground_id = gound_yolo_ID_high.ground_ID[id]
        yolo_id = gound_yolo_ID_high.yolo_ID[id]
            # 真实轨迹数据 
        traj_ground0 = Groundtruthebike[Groundtruthebike.Object_id ==ground_id]
        traj_ground0 = pd.DataFrame(traj_ground0)
            # CV输出的轨迹数据
#         CV_traj_loops = Yolov7_wt[Yolov7_wt.loopid ==loop]   
    
        CV_traj_loops = compar_traj_ebike
        traj_compare0 = CV_traj_loops[CV_traj_loops.Object_id == yolo_id]

#         mask_traj_loops = Mask_ebike_sfpf[Mask_ebike_sfpf.loopid == loop]
#         traj_compare0 = mask_traj_loops[mask_traj_loops.Object_id == yolo_id]
    #         print(traj_ground0)
    #         print(traj_ground0.World_y)
    #         min(pd.array(Groundtruthebike.World_y))
        if traj_compare0.shape[0]>=10:
            traj_ground,traj_compare = Alignment(traj_ground0,traj_compare0)

                # 提取轨迹数据
# # #       显示二维平面轨迹
            # fig1 = Trajmap(traj_ground,traj_compare)  
# #         显示时间速度对比图像   
#             fig2 = Timespeed(traj_ground,traj_compare) 
# #         显示时间加速度对比图像
#             fig3 = Timeacc(traj_ground,traj_compare) 
# #         显示时间加加速度对比图像
#             fig4 = Timejerk(traj_ground,traj_compare)
            
            RMSE = Index_rmse(traj_ground,traj_compare)
            Rmse_list.append(RMSE)
        
            # SNRRR = SNR(traj_ground,traj_compare)
            SNRRR = SNR(traj_ground,traj_compare)
            SNR_list.append(SNRRR) 

            HausdorffR = Hausdorff(traj_ground,traj_compare)
            Hausdorff_list.append(HausdorffR)
        
            #R-squared value
            Rsquared_x,Rsquared_y = RRR(traj_ground,traj_compare)
            Rsquared_x_list.append(Rsquared_x)
            Rsquared_y_list.append(Rsquared_y)
        
    A = pd.DataFrame(Rmse_list)
    RMSE_allloop.iloc[:,loop] = A
    B = pd.DataFrame(SNR_list)
    SNR_allloop.iloc[:,loop] = B
    C = pd.DataFrame(Hausdorff_list)
    Hausdorff_allloop.iloc[:,loop] = C
    RX = pd.DataFrame(Rsquared_x_list)
    Rsquared_allloop_x.iloc[:,loop] = RX
    RY = pd.DataFrame(Rsquared_y_list)
    Rsquared_allloop_y.iloc[:,loop] = RY
   
RMSE_allloop.iloc[:,1:3]=gound_yolo_ID_high.iloc[:,0:2]
SNR_allloop.iloc[:,1:3]=gound_yolo_ID_high.iloc[:,0:2]
Hausdorff_allloop.iloc[:,1:3]=gound_yolo_ID_high.iloc[:,0:2]
Rsquared_allloop_x.iloc[:,1:3]=gound_yolo_ID_high.iloc[:,0:2]
Rsquared_allloop_y.iloc[:,1:3]=gound_yolo_ID_high.iloc[:,0:2]

# print(gound_yolo_ID_high)
filepath1 = r"/content/drive/My Drive/data_set/result_compare/G_Y_H_RMSE.csv"
RMSE_allloop.to_csv(filepath1, index=False, mode='a+', header=False)
filepath2 = r"/content/drive/My Drive/data_set/result_compare/G_Y_H_SNR.csv"
SNR_allloop.to_csv(filepath2, index=False, mode='a+', header=False)
filepath3 = r"/content/drive/My Drive/data_set/result_compare/G_Y_H_Hausdorrf.csv"
Hausdorff_allloop.to_csv(filepath3, index=False, mode='a+', header=False)
filepath4 = r"/content/drive/My Drive/data_set/result_compare/G_Y_H_Rsquared_X.csv"
Rsquared_allloop_x.to_csv(filepath4, index=False, mode='a+', header=False)
filepath5 = r"/content/drive/My Drive/data_set/result_compare/G_Y_H_Rsquared_Y.csv"
Rsquared_allloop_y.to_csv(filepath5, index=False, mode='a+', header=False)